# This notebook was made with [J0EK3R](https://github.com/J0EK3R)'s [mkconnect](https://github.com/J0EK3R/mkconnect-python)

## Setting Permission
To be able to run this notebook it's nessesary to give Python the following permissions:
```bash
sudo setcap 'cap_net_raw,cap_net_admin+eip' $(which python3)
```

In [ ]:
#!/usr/bin/python

# Set up all values

import sys
import time
import concurrent.futures
import threading
import ipywidgets as widgets
from IPython.display import display


print('Script: test.ipynb')
print('Platform: ' + sys.platform)
print('Initializing..')

sys.path.append("Tracer") 
from Tracer.Tracer import Tracer
from Tracer.TracerConsole import TracerConsole

sys.path.append("Advertiser") 
# uncomment to choose advertiser
if (sys.platform == 'linux'):
    from Advertiser.AdvertiserBTSocket import AdvertiserBTSocket as Advertiser
    pass
elif (sys.platform == 'rp2'):
    from Advertiser.AdvertiserMicroPython import AdvertiserMicroPython as Advertiser
    pass
elif (sys.platform == 'win32'):
    from Advertiser.AdvertiserDummy import AdvertiserDummy as Advertiser
else:
    raise Exception('unsupported platform')

sys.path.append("MouldKing") 
from MouldKing.MouldKing import MouldKing
from MouldKingCrypt import MouldKingCrypt

# instantiate Tracer
tracer = TracerConsole()

# instantiate Advertiser
advertiser = Advertiser()
advertiser.SetTracer(tracer)

# Set Tracer for all MouldKing Hubs
MouldKing.SetTracer(tracer)
MouldKing.SetAdvertiser(advertiser)

hub = MouldKing.Module4_0.Device0

############################
### ChatGPT

# Create a global event to signal stopping the update loop
stop_event = threading.Event()

# Create an ipywidget slider to control channel speed.
speed_slider = widgets.FloatSlider(
    value=0,
    min=-1.0,
    max=1.0,
    step=0.1,
    description='Speed:',
    continuous_update=True
)

# Create a stop button widget.
stop_button = widgets.Button(
    description='Exit',
    button_style='danger'
)

# Define the callback for the stop button.
def stop_callback(b):
    stop_event.set()

stop_button.on_click(stop_callback)


##################################

print('Initializing done!')

In [ ]:
# Connect to Hub

hub.Connect()
time.sleep(5)

#tracer.TraceInfo("rawdata: " + ' '.join(f'{x:02x}' for x in rawdata))
#crypted = MouldKingCrypt.Crypt(rawdata) # get crypted data from rawdata
#tracer.TraceInfo("crypted: " + ' '.join(f'{x:02x}' for x in crypted))

In [ ]:

# Display the widgets in the notebook.
display(speed_slider, stop_button)

# Define your channel functions to accept a speed parameter.
def ch0(speed):
    hub.SetChannel(0, speed)

def ch1(speed):
    hub.SetChannel(1, speed)

def ch2(speed):
    # For channel 2, using the negative of the speed (as in your original code).
    hub.SetChannel(2, -speed)

def ch3(speed):
    hub.SetChannel(3, speed)

# This function will update the channels repeatedly until stop_event is set.
def update_channels():
    while not stop_event.is_set():
        current_speed = speed_slider.value
        # Using a thread pool to call the channel functions concurrently.
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.submit(ch0, current_speed)
            executor.submit(ch1, current_speed)
            executor.submit(ch2, current_speed)
            executor.submit(ch3, current_speed)
        time.sleep(0.5)
    
    stop_button.close()
    speed_slider.close()
    
    rawdata = hub.Stop()
    stop_event.clear()
    time.sleep(1)

# Start the update loop in a background thread.
channel_thread = threading.Thread(target=update_channels, daemon=True)
channel_thread.start()


In [ ]:
# Disconnect and stop the Advertisement Service

hub.Disconnect()
time.sleep(1)


advertiser.AdvertisementStop()
time.sleep(1)

## Emergency Stuff

In [ ]:
# Reset all Channels

rawdata = hub.Stop()
stop_event.clear()
time.sleep(1)

#tracer.TraceInfo("rawdata: " + ' '.join(f'{x:02x}' for x in rawdata))

In [ ]:
# Restore Widgets
speed_slider = widgets.FloatSlider(
    value=0,
    min=-1.0,
    max=1.0,
    step=0.1,
    description='Speed:',
    continuous_update=True
)
stop_button = widgets.Button(
    description='Exit',
    button_style='danger'
)

## Examples:

In [ ]:
# Setting specific speed to specific motor

channel = 3
speed = 0.15

rawdata = hub.SetChannel(channel, speed)

#tracer.TraceInfo("rawdata: " + ' '.join(f'{x:02x}' for x in rawdata))


In [ ]:
# First attempt at multithreading
"""
def ch0():
    rawdata = hub.SetChannel(0, 0.5)
    return

def ch1():
    rawdata = hub.SetChannel(1, 0.5)
    return

def ch2():
    rawdata = hub.SetChannel(2, -0.5)
    return

def ch3():
    rawdata = hub.SetChannel(3, 0.5)
    return


while True:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(task) for task in [ch0, ch1, ch2, ch3]]
    time.sleep(0.5)


    
#for future in concurrent.futures.as_completed(futures):
    #print(future.result())

#tracer.TraceInfo("rawdata: " + ' '.join(f'{x:02x}' for x in rawdata))
"""